# Enumerating jugglable music

TODO : Explain

TODO : Tables, OEIS.

In [94]:
from itertools import combinations, product
from typing import Callable, Iterator, Iterable, ParamSpec, TypeVar


# TODO : Silent catches could be optimized.


def is_submusic(music1: Iterable[int], music2: Iterable[int]) -> bool:
    if len(music1) != len(music2):
        return False
    for i in range(len(music1)):
        if music1[i] != 0 and music1[i] != music2[i]:
            return False
    return True


# TODO : Only works WITHOUT Silent catches. Make version that works with it (although very inneficient when iterating).
# TODO : Documenter condition aux bords : il faut que chaque balle apparaisse une fois dans les premiers max_height lancers, et dans les derniers
# TODO : test periodic, compare to previous notebook.
def is_jugglable(
    music: Iterable[int],
    max_height: int,
    nb_balls_per_note: Iterable[int],
    is_periodic: bool,
) -> bool:
    # Edge cases
    # If the music is empty, if is jugglable iff there are no balls.
    if len(music) == 0:
        return all(map(lambda x: x == 0, nb_balls_per_note))
    # If the max height is 0, the music is jugglable iff there are no balls and the music is silent
    if max_height <= 0:
        return all(map(lambda x: x == 0, nb_balls_per_note)) and all(
            map(lambda x: x == 0, music)
        )

    # Along the music, we move a frame of length max_height.
    # We check that for each note, this frame always has at least as much occurences of that note as there are balls.
    nb_balls_in_frame = [0] * len(nb_balls_per_note)
    # If the music / juggling patterns are periodic, we need to loop around, so there are more frames to consider.
    if is_periodic:
        nb_frames = len(music)
        first_frame_length = max_height
    else:
        # If max_height is higher than the music's length, we still have to examine one frame.
        if len(music) <= max_height:
            nb_frames = 1
            first_frame_length = len(music)
        else:
            nb_frames = len(music) - max_height + 1
            first_frame_length = max_height

    # Fill in the first frame.
    for i in range(first_frame_length):
        note = music[i % len(music)]
        nb_balls_in_frame[note] += 1
    # Check if that first frame is correct.
    for i in range(1, len(nb_balls_in_frame)):
        if nb_balls_in_frame[i] < nb_balls_per_note[i]:
            return False

    # Move the frame until the music's end.
    for i in range(nb_frames - 1):
        old_note = music[i % len(music)]
        new_note = music[(i + max_height) % len(music)]
        nb_balls_in_frame[old_note] -= 1
        nb_balls_in_frame[new_note] += 1
        if old_note != 0 and nb_balls_in_frame[old_note] < nb_balls_per_note[old_note]:
            return False

    return True


def iter_jugglable_musics_with_cyclic(
    music_length: int,
    max_height: int,
    nb_balls_per_notes: Iterable[int],
    is_periodic: bool,
    allow_silent_catches: bool,
) -> Iterator[tuple[int]]:
    # Without silence catches, we have an easy criteria.
    if not allow_silent_catches:
        # Generate all possible sequences of length n with the correct number of notes.
        for music in product(range(len(nb_balls_per_notes)), repeat=music_length):
            if is_jugglable(
                music=music,
                max_height=max_height,
                nb_balls_per_note=nb_balls_per_notes,
                is_periodic=is_periodic,
            ):
                yield music
    # With silent catches, we need to try to catch on every possible silents.
    # Optimization : start with musics with no 0, with 1 0, 2 0, ... and see
    # If they are jugglable (normally) or if swapping one zero to a ball makes it jugglable
    # using the jugglable musics of the previous loop.
    else:
        previous_jugglable_musics: set[tuple[int]] = set()
        new_jugglable_musics: set[tuple[int]] = set()
        for nb_silences in range(music_length):
            # Update the jugglable musics.
            previous_jugglable_musics = new_jugglable_musics
            new_jugglable_musics = set()

            nb_notes = music_length - nb_silences

            for note_positions in combinations(range(music_length), nb_notes):
                for notes_without_silence in product(
                    range(1, len(nb_balls_per_notes)), repeat=nb_notes
                ):
                    # Create the music
                    music = [0] * music_length
                    for i, note in zip(note_positions, notes_without_silence):
                        music[i] = note
                    # Check if the music is jugglable (without silent catches), if changing one silence to a note makes it jugglable.
                    if is_jugglable(
                        music=music,
                        max_height=max_height,
                        nb_balls_per_note=nb_balls_per_notes,
                        is_periodic=is_periodic,
                    ) or any(
                        map(
                            lambda jugglable_music: is_submusic(music, jugglable_music),
                            previous_jugglable_musics,
                        )
                    ):
                        immutable_music = tuple(music)
                        new_jugglable_musics.add(immutable_music)
                        yield tuple(immutable_music)


P = ParamSpec("P")
T = TypeVar("T")


def left_shift(L: tuple[T], offset: int) -> tuple[T]:
    return tuple(L[(i + offset) % len(L)] for i in range(len(L)))


# Décorateur pour supprimer des solutions qui sont des shifts cycliques d'une déjà trouvée.
def no_cyclic_shifts(func: Callable[P, Iterable[T]]) -> Callable[P, Iterable[T]]:
    def new_function(*args: P.args, **kwargs: P.kwargs):
        seen = set()
        for elem in func(*args, **kwargs):
            for i in range(1, len(elem)):
                if left_shift(elem, i) in seen:
                    break
            else:
                seen.add(tuple(elem))
                yield elem

    return new_function


@no_cyclic_shifts
def iter_jugglable_musics_up_to_cylic_shifts(
    music_length: int,
    max_height: int,
    nb_balls_per_notes: Iterable[int],
    allow_silent_catches: bool,
) -> Iterator[tuple[int]]:
    yield from iter_jugglable_musics_with_cyclic(
        music_length=music_length,
        max_height=max_height,
        nb_balls_per_notes=nb_balls_per_notes,
        is_periodic=True,
        allow_silent_catches=allow_silent_catches,
    )


def iter_jugglable_musics(
    music_length: int,
    max_height: int,
    nb_balls_per_notes: Iterable[int],
    is_periodic: bool,
    remove_cyclic_shifts: bool,
    allow_silent_catches: bool,
) -> Iterator[tuple[int]]:
    # Issue warning
    if remove_cyclic_shifts and not is_periodic:
        raise ValueError(
            "It doesn't make sense to remove cyclic shifts when the music is not periodic."
        )

    if remove_cyclic_shifts:
        yield from iter_jugglable_musics_up_to_cylic_shifts(
            music_length=music_length,
            max_height=max_height,
            nb_balls_per_notes=nb_balls_per_notes,
            allow_silent_catches=allow_silent_catches,
        )
    else:
        yield from iter_jugglable_musics_with_cyclic(
            music_length=music_length,
            max_height=max_height,
            nb_balls_per_notes=nb_balls_per_notes,
            is_periodic=is_periodic,
            allow_silent_catches=allow_silent_catches,
        )


In [95]:
def count_jugglable_musics(
    start_length: int,
    end_length: int,
    max_height: int,
    nb_balls_per_notes: Iterable[int],
    is_periodic: bool,
    remove_cyclic_shifts: bool,
    allow_silent_catches: bool,
) -> Iterator[int]:
    for i in range(start_length, end_length):
        yield len(tuple(iter_jugglable_musics(
            music_length=i,
            max_height=max_height,
            nb_balls_per_notes=nb_balls_per_notes,
            is_periodic=is_periodic,
            remove_cyclic_shifts=remove_cyclic_shifts,
            allow_silent_catches=allow_silent_catches,
        )))

In [ ]:
max_height = 3
nb_balls_per_notes = [0, 1]
is_periodic = False
remove_cyclic_shifts = False
allow_silent_catches = False
for music_length in range(20):
    print(
        len(
            tuple(
                iter_jugglable_musics_with_cyclic(
                    music_length=music_length,
                    max_height=max_height,
                    nb_balls_per_notes=nb_balls_per_notes,
                    is_periodic=is_periodic,
                    allow_silent_catches=allow_silent_catches,
                )
            )
        )
    )


0
1
3
7
13
24
44
81
149
274
504
927
1705
3136
5768
10609
19513
35890
66012


KeyboardInterrupt: 

In [102]:
is_jugglable([1, 1], 2, [0, 1], False)

True